# KAGLE COMPETITION - TITANIC ML from Disaster

Doing this competition project without any hints/help from other participants

## Libraries

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import titanic_utils

## EDA

In [66]:
train = pd.read_csv('data/train.csv', index_col='PassengerId')

colummns_wnan = train.columns[train.isna().any()].tolist()

### Embark Missing Values

Doing some research to add the missing values of 'Embarked' column:\
62: Icard, Miss. Amelie: https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html\
830: Stone, Mrs. George Nelson (Martha Evelyn): https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html

We find out both passengers were embarked in Southampton '(S)'

In [85]:
display(train.loc[train['Embarked'].isna(),:])

train['Embarked'] = train['Embarked'].fillna('S')

display(train.loc[(train.index == 62)|(train.index == 830),:])

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S


### Cabin Missing Values

Assuming same ticket = same cabin: We can input some missing 'Cabin' values by looking if some other passanger has the same 'Ticket' and the 'Cabin'

In [118]:
train.sort_values(by='Ticket', inplace=True)

#All passangers that have the same ticket as another passanger but missing cabin
same_ticket_no_cabin = train.loc[(train['Ticket'].duplicated(keep=False)) & (train['Cabin'].isna()),:]
#Passangers that have the same ticket as another passanger that has no cabin
same_ticket_yes_cabin = train.loc[(train['Ticket'].isin(same_ticket_no_cabin['Ticket'])) & (~train['Cabin'].isna()), :]
#We only need one passanger that matches the ticket number with another passanger that has no cabin
same_ticket_yes_cabin = same_ticket_yes_cabin.loc[same_ticket_yes_cabin['Ticket'].duplicated(), :]

cabin_df = pd.concat([same_ticket_no_cabin,same_ticket_yes_cabin])
cabin_df = cabin_df.sort_values(by='Ticket')
display(cabin_df)

for passenger_id, ticket in zip(same_ticket_no_cabin.index, same_ticket_no_cabin['Ticket']):
    #Search for a passanger with the same ticket and a non-missing cabin
    if ticket in np.array(same_ticket_yes_cabin[['Ticket']]):
        cabin = str(same_ticket_yes_cabin.loc[same_ticket_yes_cabin['Ticket'] == ticket, 'Cabin'])
        print(cabin)
    #cabin_df.loc[cabin_df.index == passenger_id, 'Cabin'] = cabin

#display(cabin_df)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
709,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.550,NaN,S
499,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.550,C22 C26,S
298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.550,C22 C26,S
36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.000,NaN,S
384,1,1,"Holverson, Mrs. Alexander Oskar (Mary Aline To...",female,35.0,1,0,113789,52.000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.450,NaN,S
437,0,3,"Ford, Miss. Doolina Margaret ""Daisy""",female,21.0,2,2,W./C. 6608,34.375,NaN,S
87,0,3,"Ford, Mr. William Neal",male,16.0,1,3,W./C. 6608,34.375,NaN,S


PassengerId
298    C22 C26
499    C22 C26
Name: Cabin, dtype: object
PassengerId
582    C68
551    C70
Name: Cabin, dtype: object
PassengerId
680    B51 B53 B55
Name: Cabin, dtype: object
PassengerId
701    C62 C64
Name: Cabin, dtype: object
PassengerId
701    C62 C64
Name: Cabin, dtype: object
PassengerId
326    C32
Name: Cabin, dtype: object
